In [17]:
import pandas as pd
import pyarrow as pa

In [18]:
# Получаем все файлы
train = pd.read_csv('train_events.csv')
video = pd.read_csv("video_info_v3.csv")
target = pd.read_csv("train_targets.csv")
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid
0,2024-06-01 06:40:58+03:00,Chelyabinsk,desktop,browser,Windows,Yandex Browser,1883,video_133074,10067243
1,2024-06-01 19:33:24+03:00,Bashkortostan Republic,smartphone,mobile app,Android,Rutube,512,video_362960,10245341
2,2024-06-01 21:30:43+03:00,St.-Petersburg,desktop,browser,Windows,Chrome,5647,video_96775,10894333
3,2024-06-01 23:03:42+03:00,Moscow,smartphone,mobile app,Android,Rutube,1521,video_161610,10029092
4,2024-06-01 22:48:09+03:00,Moscow,smartphone,mobile app,Android,Rutube,71,video_116245,10452976
...,...,...,...,...,...,...,...,...,...
1759611,2024-06-30 17:08:36+03:00,Moscow,smartphone,mobile app,Android,Rutube,4230,video_162776,10026914
1759612,2024-06-30 01:20:16+03:00,Moscow,smartphone,browser,Android,Firefox Mobile,382,video_316157,10417567
1759613,2024-06-30 22:37:04+03:00,St.-Petersburg,desktop,browser,Windows,Microsoft Edge,1342,video_83304,10009094
1759614,2024-06-30 11:33:09+03:00,St.-Petersburg,smartphone,mobile app,Android,Rutube,801,video_132769,10574374


In [19]:
# Редактируем поле регион для последующего энкодинга
train['region'] = train['region'].replace(r"[’'`]", '', regex=True)

In [20]:
# Соединяем датасеты
train = train.merge(target, on='viewer_uid', how='inner')
train = train.merge(video, how='inner', on='rutube_video_id')


In [21]:
from datetime import timedelta
# Приводим данные к UTC
train['event_timestamp'] = pd.to_datetime(train['event_timestamp'])
train['event_timestamp'] = train['event_timestamp'] - timedelta(hours=3) # 6-12 утро 13-18 день 19-0 вечер 1-5 ночь


In [22]:
# Все регионы и их часовые пояса
time_zones = {
    'Tatarstan Republic': 3,
    'Bashkortostan Republic': 5,
    'Novosibirsk Oblast': 7,
    'Moscow': 3,
    'Moscow Oblast': 3,
    'Omsk Oblast': 7,
    'Chuvashia': 3,
    'Krasnoyarsk Krai': 7,
    'Chelyabinsk': 5,
    'Kamchatka': 11,
    'Nizhny Novgorod Oblast': 3,
    'Krasnodar Krai': 3,
    'Volgograd Oblast': 3,
    'Kaliningrad Oblast': 2,
    'Kuzbass': 7,
    'Stavropol Kray': 3,
    'Samara Oblast': 4,
    'Amur Oblast': 9,
    'Sverdlovsk Oblast': 5,
    'St.-Petersburg': 3,
    'Yamalo-Nenets': 5,
    'Orenburg Oblast': 5,
    'Khanty-Mansia': 5,
    'Kaluga Oblast': 3,
    'Tomsk Oblast': 7,
    'Novgorod Oblast': 3,
    'Arkhangelskaya': 3,
    'North Ossetia–Alania': 3,
    'Kursk Oblast': 3,
    "Leningradskaya Oblast'": 3,
    'Krasnoyarskiy': 7,
    'Ivanovo Oblast': 3,
    'Altay Kray': 7,
    'Kurgan Oblast': 5,
    'Kostroma Oblast': 3,
    'Bryansk Oblast': 3,
    'Dagestan': 3,
    'Lipetsk Oblast': 3,
    'Vladimir Oblast': 3,
    'Kirov Oblast': 3,
    'Khabarovsk': 9,
    'Tambov Oblast': 3,
    'Chukotka': 12,
    'Voronezh Oblast': 3,
    'Sverdlovsk': 5,
    'Tula Oblast': 3,
    'Krasnodarskiy': 3,
    'Irkutsk Oblast': 8,
    'Saratov Oblast': 4,
    'Khakasiya Republic': 7,
    'Penza': 3,
    'Perm Krai': 5,
    'Oryol oblast': 3,
    'Vladimir': 3,
    'Smolensk Oblast': 3,
    'Penza Oblast': 3,
    'Mordoviya Republic': 3,
    'Tyumen’ Oblast': 5,
    'Sakha': 9,
    'Primorye': 10,
    'Zabaykalskiy (Transbaikal) Kray': 9,
    'Vologda Oblast': 3,
    'Yaroslavl Oblast': 3,
    'Crimea': 3,
    'Rostov': 3,
    'Ryazan Oblast': 3,
    'Perm': 5,
    'Chechnya': 3,
    'Udmurtiya Republic': 3,
    'Tver Oblast': 3,
    'Buryatiya Republic': 8,
    'Belgorod Oblast': 3,
    'Kaluga': 3,
    'Astrakhan Oblast': 3,
    'Karelia': 3,
    'Murmansk': 3,
    'Adygeya Republic': 3,
    'Kemerovo Oblast': 7,
    'Mariy-El Republic': 3,
    'Kursk': 3,
    'Saratovskaya Oblast': 4,
    'Sakhalin Oblast': 10,
    'Ivanovo': 3,
    'Tyumen Oblast': 5,
    'Stavropol’ Kray': 3,
    'Voronezj': 3,
    'Karachayevo-Cherkesiya Republic': 3,
    'Kabardino-Balkariya Republic': 3,
    'Ulyanovsk': 4,
    'North Ossetia': 3,
    'Komi': 3,
    'Smolensk': 3,
    'Tver’ Oblast': 3,
    'Sebastopol City': 3,
    'Pskov Oblast': 3,
    'Tula': 3,
    'Orel Oblast': 3,
    'Jaroslavl': 3,
    'Tambov': 3,
    'Kalmykiya Republic': 3,
    'Primorskiy (Maritime) Kray': 10,
    'Altai': 7,
    'Magadan Oblast': 10,
    'Vologda': 3,
    'Tyva Republic': 7,
    'Nenets': 3,
    'Smolenskaya Oblast': 3,
    'Jewish Autonomous Oblast': 10,
    'Astrakhan': 3,
    'Ingushetiya Republic': 3,
    'Kirov': 3,
    'Transbaikal Territory': 9,
    'Omsk': 7,
    'Kaliningrad': 2,
    'Stavropol Krai': 3,
    'Arkhangelsk Oblast': 3,
    'Leningradskaya Oblast': 3
}

In [23]:
# Делаем колонку со смещением по временным зонам относительно региона пользователя
train['timezone_offset'] = train['region'].map(time_zones)

In [24]:
# Вычисляем время относительно часового пояса региона пользователя
train['event_timestamp_timezone'] = train['event_timestamp'] + pd.to_timedelta(train['timezone_offset'], unit='h')

In [25]:
# Вычисляем в какую часть дня пользователь просматривал видео
train['morning'] = 0
train['day'] = 0
train['evening'] = 0
train['night'] = 0
train.loc[train['event_timestamp_timezone'].dt.hour.between(6, 12, inclusive='left'), 'morning'] = 1
train.loc[train['event_timestamp_timezone'].dt.hour.between(13, 18, inclusive='left'), 'day'] = 1
train.loc[train['event_timestamp_timezone'].dt.hour.between(19, 23, inclusive='left'), 'evening'] = 1
train.loc[train['event_timestamp_timezone'].dt.hour.between(0, 5, inclusive='both'), 'night'] = 1

In [26]:
# Делаем Label Encoding категорий видео
category = video['category'].unique()
cat_dict = {c: c_id for c_id, c in enumerate(category)}
train['category'] = train['category'].map(cat_dict)

In [27]:
# Делаем Label Encoding типов устройства
devices = train['ua_device_type'].unique()
device_dict = {dev: dev_i for dev_i, dev in enumerate(devices)}
train['ua_device_type'] = train['ua_device_type'].map(device_dict)

In [28]:
# Делаем Label Encoding регионов
region_unique = train['region'].unique()
region_dict = {reg: reg_i for reg_i, reg in enumerate(region_unique)}
train['region'] = train['region'].map(region_dict)

In [29]:
# Пересчитываем время видео из секунды в миллисекунды
train['duration'] = train['duration'] / 1000
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,age,...,category,duration,author_id,label,timezone_offset,event_timestamp_timezone,morning,day,evening,night
0,2024-06-01 03:40:58+03:00,0,0,browser,Windows,Yandex Browser,1883,video_133074,10067243,20,...,16,2456.534,1009219,31,5,2024-06-01 08:40:58+03:00,1,0,0,0
1,2024-06-01 16:33:24+03:00,1,1,mobile app,Android,Rutube,512,video_362960,10245341,40,...,29,519.211,1006760,30,5,2024-06-01 21:33:24+03:00,0,0,1,0
2,2024-06-01 18:30:43+03:00,2,0,browser,Windows,Chrome,5647,video_96775,10894333,23,...,16,5518.280,1009257,31,3,2024-06-01 21:30:43+03:00,0,0,1,0
3,2024-06-01 20:03:42+03:00,3,1,mobile app,Android,Rutube,1521,video_161610,10029092,41,...,4,1522.069,1058671,42,3,2024-06-01 23:03:42+03:00,0,0,0,0
4,2024-06-01 19:48:09+03:00,3,1,mobile app,Android,Rutube,71,video_116245,10452976,38,...,12,1249.920,1020020,30,3,2024-06-01 22:48:09+03:00,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759611,2024-06-30 14:08:36+03:00,3,1,mobile app,Android,Rutube,4230,video_162776,10026914,32,...,1,4480.915,1095392,8,3,2024-06-30 17:08:36+03:00,0,1,0,0
1759612,2024-06-29 22:20:16+03:00,3,1,browser,Android,Firefox Mobile,382,video_316157,10417567,48,...,10,320.134,1033688,12,3,2024-06-30 01:20:16+03:00,0,0,0,1
1759613,2024-06-30 19:37:04+03:00,2,0,browser,Windows,Microsoft Edge,1342,video_83304,10009094,28,...,16,3125.675,1009219,31,3,2024-06-30 22:37:04+03:00,0,0,1,0
1759614,2024-06-30 08:33:09+03:00,2,1,mobile app,Android,Rutube,801,video_132769,10574374,43,...,16,2401.283,1009219,31,3,2024-06-30 11:33:09+03:00,1,0,0,0


In [30]:
# Считаем процент того, сколько по времени юзер смотрел видео
train['percent'] = train['total_watchtime'] / train['duration']

In [31]:
# Фильтрация по проценту
train = train.drop(train[train['percent'] > 1.1].index)

In [32]:
Q1 = train['total_watchtime'].quantile(0.4)
Q3 = train['total_watchtime'].quantile(0.9)

In [33]:
Q3

4766.0

In [34]:
mask = train['total_watchtime'].between(Q1, Q3)
train = train.loc[mask]

In [35]:
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,age,...,duration,author_id,label,timezone_offset,event_timestamp_timezone,morning,day,evening,night,percent
0,2024-06-01 03:40:58+03:00,0,0,browser,Windows,Yandex Browser,1883,video_133074,10067243,20,...,2456.534,1009219,31,5,2024-06-01 08:40:58+03:00,1,0,0,0,0.766527
3,2024-06-01 20:03:42+03:00,3,1,mobile app,Android,Rutube,1521,video_161610,10029092,41,...,1522.069,1058671,42,3,2024-06-01 23:03:42+03:00,0,0,0,0,0.999298
6,2024-06-01 19:14:37+03:00,5,0,browser,Windows,Microsoft Edge,3882,video_53642,10058831,46,...,3983.960,1009257,31,3,2024-06-01 22:14:37+03:00,0,0,1,0,0.974407
7,2024-06-01 05:42:11+03:00,6,1,mobile app,Android,Rutube,2480,video_114660,10084832,26,...,2550.593,1084744,35,3,2024-06-01 08:42:11+03:00,1,0,0,0,0.972323
8,2024-06-01 08:09:51+03:00,7,1,mobile app,Android,Rutube,3140,video_425474,10439246,22,...,3148.075,1095392,8,7,2024-06-01 15:09:51+03:00,0,1,0,0,0.997435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759601,2024-06-30 05:32:04+03:00,27,1,mobile app,Android,Rutube,3061,video_333507,10194039,37,...,3224.000,1009257,31,3,2024-06-30 08:32:04+03:00,1,0,0,0,0.949442
1759606,2024-06-30 15:18:44+03:00,24,1,mobile app,Android,Rutube,2381,video_246846,10034725,21,...,3800.865,1036089,40,9,2024-07-01 00:18:44+03:00,0,0,0,1,0.626436
1759607,2024-06-30 12:58:04+03:00,82,0,browser,Windows,Chrome,1661,video_86744,10015493,33,...,2773.200,1002256,30,5,2024-06-30 17:58:04+03:00,0,1,0,0,0.598947
1759611,2024-06-30 14:08:36+03:00,3,1,mobile app,Android,Rutube,4230,video_162776,10026914,32,...,4480.915,1095392,8,3,2024-06-30 17:08:36+03:00,0,1,0,0,0.944004


In [36]:
train = train.drop(['title'], axis=1)

In [37]:
client_type = train['ua_client_type'].unique()
client_type_dict = {cl: cl_id for cl_id, cl in enumerate(client_type)}
train['ua_client_type'] = train['ua_client_type'].map(client_type_dict)

In [38]:
train = train.drop(train[train['ua_client_type']==2].index)

In [39]:
train['ua_os'] = train['ua_os'].fillna("Undefined")

In [40]:
oss = train['ua_os'].unique()
os_dict = {os: os_i for os_i, os in enumerate(oss)}
train['ua_os'] = train['ua_os'].map(os_dict)
train

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,age,...,duration,author_id,label,timezone_offset,event_timestamp_timezone,morning,day,evening,night,percent
0,2024-06-01 03:40:58+03:00,0,0,0,0,Yandex Browser,1883,video_133074,10067243,20,...,2456.534,1009219,31,5,2024-06-01 08:40:58+03:00,1,0,0,0,0.766527
3,2024-06-01 20:03:42+03:00,3,1,1,1,Rutube,1521,video_161610,10029092,41,...,1522.069,1058671,42,3,2024-06-01 23:03:42+03:00,0,0,0,0,0.999298
6,2024-06-01 19:14:37+03:00,5,0,0,0,Microsoft Edge,3882,video_53642,10058831,46,...,3983.960,1009257,31,3,2024-06-01 22:14:37+03:00,0,0,1,0,0.974407
7,2024-06-01 05:42:11+03:00,6,1,1,1,Rutube,2480,video_114660,10084832,26,...,2550.593,1084744,35,3,2024-06-01 08:42:11+03:00,1,0,0,0,0.972323
8,2024-06-01 08:09:51+03:00,7,1,1,1,Rutube,3140,video_425474,10439246,22,...,3148.075,1095392,8,7,2024-06-01 15:09:51+03:00,0,1,0,0,0.997435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759601,2024-06-30 05:32:04+03:00,27,1,1,1,Rutube,3061,video_333507,10194039,37,...,3224.000,1009257,31,3,2024-06-30 08:32:04+03:00,1,0,0,0,0.949442
1759606,2024-06-30 15:18:44+03:00,24,1,1,1,Rutube,2381,video_246846,10034725,21,...,3800.865,1036089,40,9,2024-07-01 00:18:44+03:00,0,0,0,1,0.626436
1759607,2024-06-30 12:58:04+03:00,82,0,0,0,Chrome,1661,video_86744,10015493,33,...,2773.200,1002256,30,5,2024-06-30 17:58:04+03:00,0,1,0,0,0.598947
1759611,2024-06-30 14:08:36+03:00,3,1,1,1,Rutube,4230,video_162776,10026914,32,...,4480.915,1095392,8,3,2024-06-30 17:08:36+03:00,0,1,0,0,0.944004


In [41]:
client_names = train['ua_client_name'].unique()
client_name_dict = {cl_name: cl_name_id for cl_name_id, cl_name in enumerate(client_names)}
train['ua_client_name'] = train['ua_client_name'].map(client_name_dict)

In [42]:
from sklearn.cluster import KMeans
# Делаем группы устройств
kmeans = KMeans(n_clusters=10, max_iter=1000).fit(train[['ua_device_type', 'ua_client_type', 'ua_os', 'ua_client_name']])

In [43]:
train['device'] = kmeans.labels_

In [44]:
category_cols = [f"category_{i}" for i in range(40)]
label_cols = [f"label_{i}" for i in range(50)]
device_cols = [f"device_{i}" for i in range(len(train['device'].unique()))]
authors_cols = [f"author_{i}" for i in range(len(video['author_id'].unique()))]

In [45]:
%%time
def get_mode(series):
    return series.mode().iloc[0] if not series.mode().empty else None
# Аггрегируем данные, генерируем новые фичи
aggregated = train.groupby('viewer_uid').agg({
    'event_timestamp_timezone': ['count', 'min', 'max', 'mean'],
    'region': get_mode,
    'label': get_mode,
    'category': get_mode,
    'total_watchtime': ['mean', 'max', 'min', 'sum'],
    'device': get_mode,
    'author_id': get_mode,
    'rutube_video_id': get_mode,
    'percent': ['mean', 'min', 'max', 'sum'],
    'duration': ['min', 'max', 'sum', 'mean']
})
aggregated

CPU times: total: 1min 27s
Wall time: 1min 27s


event_timestamp_timezone                            \
                              count                       min   
viewer_uid                                                      
10000001                         11 2024-06-01 11:54:32+03:00   
10000002                          3 2024-06-07 20:14:13+03:00   
10000004                          3 2024-06-16 14:47:45+03:00   
10000007                          4 2024-06-08 19:10:14+03:00   
10000012                          3 2024-06-17 14:08:33+03:00   
...                             ...                       ...   
11140666                          2 2024-06-30 21:54:30+03:00   
11140667                          1 2024-06-30 21:36:13+03:00   
11140684                          1 2024-06-30 22:45:45+03:00   
11140718                          1 2024-06-30 23:29:09+03:00   
11140804                          1 2024-07-01 00:49:44+03:00   

                                                                          \
                                 max                                mean   
viewer_uid                                                                 
10000001   2024-06-22 21:53:33+03:00 2024-06-06 16:33:56.636363776+03:00   
10000002   2024-06-08 00:05:26+03:00 2024-06-07 21:52:56.333333248+03:00   
10000004   2024-06-26 09:17:33+03:00 2024-06-21 06:23:30.999999744+03:00   
10000007   2024-06-21 05:58:03+03:00    2024-06-14 03:31:55.500000+03:00   
10000012   2024-06-17 16:12:29+03:00 2024-06-17 15:14:02.999999744+03:00   
...                              ...                                 ...   
11140666   2024-06-30 22:48:01+03:00    2024-06-30 22:21:15.500000+03:00   
11140667   2024-06-30 21:36:13+03:00           2024-06-30 21:36:13+03:00   
11140684   2024-06-30 22:45:45+03:00           2024-06-30 22:45:45+03:00   
11140718   2024-06-30 23:29:09+03:00           2024-06-30 23:29:09+03:00   
11140804   2024-07-01 00:49:44+03:00           2024-07-01 00:49:44+03:00   

             region    label category total_watchtime              ...  \
           get_mode get_mode get_mode            mean   max   min  ...   
viewer_uid                                                         ...   
10000001          3       31       16     3446.181818  4622  1548  ...   
10000002          3       32        1     1297.000000  1311  1270  ...   
10000004          3       30        1     2556.000000  2690  2361  ...   
10000007          4       11       16     2651.000000  2789  2260  ...   
10000012          1       26        1     2832.333333  3284  2192  ...   
...             ...      ...      ...             ...   ...   ...  ...   
11140666          3       11        1     2903.500000  3230  2577  ...   
11140667         13        8        2     2794.000000  2794  2794  ...   
11140684         39       31       16     4311.000000  4311  4311  ...   
11140718          3       47       16     2534.000000  2534  2534  ...   
11140804         22       44       16     3256.000000  3256  3256  ...   

           author_id rutube_video_id   percent                                \
            get_mode        get_mode      mean       min       max       sum   
viewer_uid                                                                     
10000001     1009257    video_204107  0.747531  0.356039  0.999892  8.222836   
10000002     1065557    video_148064  0.996139  0.994275  0.998969  2.988418   
10000004     1010000    video_342531  0.503358  0.499478  0.509045  1.510074   
10000007     1009223    video_120267  0.967032  0.897046  0.999123  3.868127   
10000012     1043618    video_191142  0.932787  0.905337  0.963500  2.798361   
...              ...             ...       ...       ...       ...       ...   
11140666     1043618    video_208085  0.903522  0.885864  0.921181  1.807045   
11140667     1036433     video_60937  0.783002  0.783002  0.783002  0.783002   
11140684     1009257    video_115806  0.639330  0.639330  0.639330  0.639330   
11140718     1009257     video_40997  

In [46]:
aggregated = aggregated.reset_index(drop=False)
aggregated

viewer_uid event_timestamp_timezone                            \
                                     count                       min   
0        10000001                       11 2024-06-01 11:54:32+03:00   
1        10000002                        3 2024-06-07 20:14:13+03:00   
2        10000004                        3 2024-06-16 14:47:45+03:00   
3        10000007                        4 2024-06-08 19:10:14+03:00   
4        10000012                        3 2024-06-17 14:08:33+03:00   
...           ...                      ...                       ...   
108541   11140666                        2 2024-06-30 21:54:30+03:00   
108542   11140667                        1 2024-06-30 21:36:13+03:00   
108543   11140684                        1 2024-06-30 22:45:45+03:00   
108544   11140718                        1 2024-06-30 23:29:09+03:00   
108545   11140804                        1 2024-07-01 00:49:44+03:00   

                                                                       region  \
                             max                                mean get_mode   
0      2024-06-22 21:53:33+03:00 2024-06-06 16:33:56.636363776+03:00        3   
1      2024-06-08 00:05:26+03:00 2024-06-07 21:52:56.333333248+03:00        3   
2      2024-06-26 09:17:33+03:00 2024-06-21 06:23:30.999999744+03:00        3   
3      2024-06-21 05:58:03+03:00    2024-06-14 03:31:55.500000+03:00        4   
4      2024-06-17 16:12:29+03:00 2024-06-17 15:14:02.999999744+03:00        1   
...                          ...                                 ...      ...   
108541 2024-06-30 22:48:01+03:00    2024-06-30 22:21:15.500000+03:00        3   
108542 2024-06-30 21:36:13+03:00           2024-06-30 21:36:13+03:00       13   
108543 2024-06-30 22:45:45+03:00           2024-06-30 22:45:45+03:00       39   
108544 2024-06-30 23:29:09+03:00           2024-06-30 23:29:09+03:00        3   
108545 2024-07-01 00:49:44+03:00           2024-07-01 00:49:44+03:00       22   

          label category total_watchtime        ... author_id rutube_video_id  \
       get_mode get_mode            mean   max  ...  get_mode        get_mode   
0            31       16     3446.181818  4622  ...   1009257    video_204107   
1            32        1     1297.000000  1311  ...   1065557    video_148064   
2            30        1     2556.000000  2690  ...   1010000    video_342531   
3            11       16     2651.000000  2789  ...   1009223    video_120267   
4            26        1     2832.333333  3284  ...   1043618    video_191142   
...         ...      ...             ...   ...  ...       ...             ...   
108541       11        1     2903.500000  3230  ...   1043618    video_208085   
108542        8        2     2794.000000  2794  ...   1036433     video_60937   
108543       31       16     4311.000000  4311  ...   1009257    video_115806   
108544       47       16     2534.000000  2534  ...   1009257     video_40997   
108545       44       16     3256.000000  3256  ...   1009257     video_80057   

         percent                                duration                       \
            mean       min       max       sum       min       max        sum   
0       0.747531  0.356039  0.999892  8.222836  4133.720  5917.000  50493.080   
1       0.996139  0.994275  0.998969  2.988418  1277.313  1316.353   3906.019   
2       0.503358  0.499478  0.509045  1.510074  4726.934  5363.360  15231.294   
3       0.967032  0.897046  0.999123  3.868127  2519.379  2859.689  10945.762   
4       0.932787  0.905337  0.963500  2.798361  2275.040  3627.378   9152.467   
...          ...       ...       ...       ...       ...       ...        ...   
108541  0.903522  0.885864  0.921181  1.807045  2909.025  3506.369   6415.394   
108542  0.783002  0.783002  0.783002  0.783002  3568.320  3568.320   3568.320   
108543  0.639330  0.639330  0.639330  0.639330  6743.000  6743.000   6743.000   
108544  0.910319  0.910319  0.910319  0.910319  2783.640  2783.640   27

In [47]:
aggregated.columns = ['_'.join(filter(None, col)).strip() for col in aggregated.columns]
aggregated.columns

Index(['viewer_uid', 'event_timestamp_timezone_count',
       'event_timestamp_timezone_min', 'event_timestamp_timezone_max',
       'event_timestamp_timezone_mean', 'region_get_mode', 'label_get_mode',
       'category_get_mode', 'total_watchtime_mean', 'total_watchtime_max',
       'total_watchtime_min', 'total_watchtime_sum', 'device_get_mode',
       'author_id_get_mode', 'rutube_video_id_get_mode', 'percent_mean',
       'percent_min', 'percent_max', 'percent_sum', 'duration_min',
       'duration_max', 'duration_sum', 'duration_mean'],
      dtype='object')

In [48]:
# Приводим время к нужному типу
aggregated['event_timestamp_timezone_min'] = aggregated['event_timestamp_timezone_min'].apply(lambda x: x.timestamp())
aggregated['event_timestamp_timezone_max'] = aggregated['event_timestamp_timezone_max'].apply(lambda x: x.timestamp())
aggregated['event_timestamp_timezone_mean'] = aggregated['event_timestamp_timezone_mean'].apply(lambda x: x.timestamp())
aggregated

,viewer_uid,event_timestamp_timezone_count,event_timestamp_timezone_min,event_timestamp_timezone_max,event_timestamp_timezone_mean,region_get_mode,label_get_mode,category_get_mode,total_watchtime_mean,total_watchtime_max,...,author_id_get_mode,rutube_video_id_get_mode,percent_mean,percent_min,percent_max,percent_sum,duration_min,duration_max,duration_sum,duration_mean
0,10000001,11,1.717232e+09,1.719082e+09,1.717681e+09,3,31,16,3446.181818,4622,...,1009257,video_204107,0.747531,0.356039,0.999892,8.222836,4133.720,5917.000,50493.080,4590.280000
1,10000002,3,1.717780e+09,1.717794e+09,1.717786e+09,3,32,1,1297.000000,1311,...,1065557,video_148064,0.996139,0.994275,0.998969,2.988418,1277.313,1316.353,3906.019,1302.006333
2,10000004,3,1.718538e+09,1.719383e+09,1.718940e+09,3,30,1,2556.000000,2690,...,1010000,video_342531,0.503358,0.499478,0.509045,1.510074,4726.934,5363.360,15231.294,5077.098000
3,10000007,4,1.717863e+09,1.718939e+09,1.718325e+09,4,11,16,2651.000000,2789,...,1009223,video_120267,0.967032,0.897046,0.999123,3.868127,2519.379,2859.689,10945.762,2736.440500
4,10000012,3,1.718623e+09,1.718630e+09,1.718626e+09,1,26,1,2832.333333,3284,...,1043618,video_191142,0.932787,0.905337,0.963500,2.798361,2275.040,3627.378,9152.467,3050.822333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108541,11140666,2,1.719774e+09,1.719777e+09,1.719775e+09,3,11,1,2903.500000,3230,...,1043618,video_208085,0.903522,0.885864,0.921181,1.807045,2909.025,3506.369,6415.394,3207.697000
108542,11140667,1,1.719773e+09,1.719773e+09,1.719773e+09,13,8,2,2794.000000,2794,...,1036433,video_60937,0.783002,0.783002,0.783002,0.783002,3568.320,3568.320,3568.320,3568.320000
108543,11140684,1,1.719777e+09,1.719777e+09,1.719777e+09,39,31,16,4311.000000,4311,...,1009257,video_115806,0.639330,0.639330,0.639330,0.639330,6743.000,6743.000,6743.000,6743.000000
108544,11140718,1,1.719779e+09,1.719779e+09,1.719779e+09,3,47,16,2534.000000,2534,...,1009257,video_40997,0.910319,0.910319,0.910319,0.910319,2783.640,2783.640,2783.640,2783.640000


In [49]:
aggregated = aggregated.merge(target, on='viewer_uid', how='inner')

In [50]:
"""label_counts = train.pivot_table(index='viewer_uid', columns='label', aggfunc='size', fill_value=0)

for la in range(len(label_cols)):
    if la not in label_counts.columns:
        label_counts[la] = 0

label_counts = label_counts.reset_index().reindex(columns=['viewer_uid'] + list(range(len(label_cols))))

label_counts.columns = ['viewer_uid'] + label_cols

label_df = label_counts"""

"label_counts = train.pivot_table(index='viewer_uid', columns='label', aggfunc='size', fill_value=0)\n\nfor la in range(len(label_cols)):\n    if la not in label_counts.columns:\n        label_counts[la] = 0\n\nlabel_counts = label_counts.reset_index().reindex(columns=['viewer_uid'] + list(range(len(label_cols))))\n\nlabel_counts.columns = ['viewer_uid'] + label_cols\n\nlabel_df = label_counts"

In [51]:
"""category_counts = train.pivot_table(index='viewer_uid', columns='category', aggfunc='size', fill_value=0)

for category in range(len(category_cols)):
    if category not in category_counts.columns:
        category_counts[category] = 0

category_counts = category_counts.reset_index().reindex(columns=['viewer_uid'] + list(range(len(category_cols))))

category_counts.columns = ['viewer_uid'] + category_cols

cat_df = category_counts"""

"category_counts = train.pivot_table(index='viewer_uid', columns='category', aggfunc='size', fill_value=0)\n\nfor category in range(len(category_cols)):\n    if category not in category_counts.columns:\n        category_counts[category] = 0\n\ncategory_counts = category_counts.reset_index().reindex(columns=['viewer_uid'] + list(range(len(category_cols))))\n\ncategory_counts.columns = ['viewer_uid'] + category_cols\n\ncat_df = category_counts"

In [52]:
"""device_counts = train.pivot_table(index='viewer_uid', columns='device', aggfunc='size', fill_value=0)

for device in range(len(device_cols)):
    if device not in device_counts.columns:
        device_counts[device] = 0

device_counts = device_counts.reset_index().reindex(columns=['viewer_uid'] + list(range(len(device_cols))))

device_counts.columns = ['viewer_uid'] + device_cols

device_df = device_counts"""

"device_counts = train.pivot_table(index='viewer_uid', columns='device', aggfunc='size', fill_value=0)\n\nfor device in range(len(device_cols)):\n    if device not in device_counts.columns:\n        device_counts[device] = 0\n\ndevice_counts = device_counts.reset_index().reindex(columns=['viewer_uid'] + list(range(len(device_cols))))\n\ndevice_counts.columns = ['viewer_uid'] + device_cols\n\ndevice_df = device_counts"

In [53]:
t = pa.Table.from_pandas(train)

In [54]:
# Считаем кол-во просмотров по части дня
time_sum = t.select(['viewer_uid', 'morning', 'day','evening', 'night']).group_by(['viewer_uid']).aggregate([('morning', "sum"), ('day', "sum"), ('evening', "sum"), ('night', "sum")])
time_sum = time_sum.to_pandas()

In [55]:
#aggregated = aggregated.merge(cat_df, how='inner', on='viewer_uid')

In [56]:
#aggregated = aggregated.merge(label_df, how='inner', on='viewer_uid')

In [57]:
#aggregated = aggregated.merge(device_df, how='inner', on='viewer_uid')

In [58]:
aggregated = aggregated.merge(time_sum, how='inner', on='viewer_uid')

In [59]:
aggregated

,viewer_uid,event_timestamp_timezone_count,event_timestamp_timezone_min,event_timestamp_timezone_max,event_timestamp_timezone_mean,region_get_mode,label_get_mode,category_get_mode,total_watchtime_mean,total_watchtime_max,...,duration_max,duration_sum,duration_mean,age,sex,age_class,morning_sum,day_sum,evening_sum,night_sum
0,10000001,11,1.717232e+09,1.719082e+09,1.717681e+09,3,31,16,3446.181818,4622,...,5917.000,50493.080,4590.280000,40,female,2,1,0,9,1
1,10000002,3,1.717780e+09,1.717794e+09,1.717786e+09,3,32,1,1297.000000,1311,...,1316.353,3906.019,1302.006333,44,male,3,0,0,2,1
2,10000004,3,1.718538e+09,1.719383e+09,1.718940e+09,3,30,1,2556.000000,2690,...,5363.360,15231.294,5077.098000,36,male,2,1,1,1,0
3,10000007,4,1.717863e+09,1.718939e+09,1.718325e+09,4,11,16,2651.000000,2789,...,2859.689,10945.762,2736.440500,40,male,2,2,0,1,1
4,10000012,3,1.718623e+09,1.718630e+09,1.718626e+09,1,26,1,2832.333333,3284,...,3627.378,9152.467,3050.822333,31,male,2,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108541,11140666,2,1.719774e+09,1.719777e+09,1.719775e+09,3,11,1,2903.500000,3230,...,3506.369,6415.394,3207.697000,37,male,2,0,0,2,0
108542,11140667,1,1.719773e+09,1.719773e+09,1.719773e+09,13,8,2,2794.000000,2794,...,3568.320,3568.320,3568.320000,24,female,1,0,0,1,0
108543,11140684,1,1.719777e+09,1.719777e+09,1.719777e+09,39,31,16,4311.000000,4311,...,6743.000,6743.000,6743.000000,47,female,3,0,0,1,0
108544,11140718,1,1.719779e+09,1.719779e+09,1.719779e+09,3,47,16,2534.000000,2534,...,2783.640,2783.640,2783.640000,37,male,2,0,0,0,0


In [60]:
# Label Encoding по видео
videos = video['rutube_video_id'].unique()
video_dict = {v: v_id for v_id, v in enumerate(videos)}

In [61]:
aggregated['sex'] = aggregated['sex'].map({'male': 1, 'female': 0})
aggregated

,viewer_uid,event_timestamp_timezone_count,event_timestamp_timezone_min,event_timestamp_timezone_max,event_timestamp_timezone_mean,region_get_mode,label_get_mode,category_get_mode,total_watchtime_mean,total_watchtime_max,...,duration_max,duration_sum,duration_mean,age,sex,age_class,morning_sum,day_sum,evening_sum,night_sum
0,10000001,11,1.717232e+09,1.719082e+09,1.717681e+09,3,31,16,3446.181818,4622,...,5917.000,50493.080,4590.280000,40,0,2,1,0,9,1
1,10000002,3,1.717780e+09,1.717794e+09,1.717786e+09,3,32,1,1297.000000,1311,...,1316.353,3906.019,1302.006333,44,1,3,0,0,2,1
2,10000004,3,1.718538e+09,1.719383e+09,1.718940e+09,3,30,1,2556.000000,2690,...,5363.360,15231.294,5077.098000,36,1,2,1,1,1,0
3,10000007,4,1.717863e+09,1.718939e+09,1.718325e+09,4,11,16,2651.000000,2789,...,2859.689,10945.762,2736.440500,40,1,2,2,0,1,1
4,10000012,3,1.718623e+09,1.718630e+09,1.718626e+09,1,26,1,2832.333333,3284,...,3627.378,9152.467,3050.822333,31,1,2,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108541,11140666,2,1.719774e+09,1.719777e+09,1.719775e+09,3,11,1,2903.500000,3230,...,3506.369,6415.394,3207.697000,37,1,2,0,0,2,0
108542,11140667,1,1.719773e+09,1.719773e+09,1.719773e+09,13,8,2,2794.000000,2794,...,3568.320,3568.320,3568.320000,24,0,1,0,0,1,0
108543,11140684,1,1.719777e+09,1.719777e+09,1.719777e+09,39,31,16,4311.000000,4311,...,6743.000,6743.000,6743.000000,47,0,3,0,0,1,0
108544,11140718,1,1.719779e+09,1.719779e+09,1.719779e+09,3,47,16,2534.000000,2534,...,2783.640,2783.640,2783.640000,37,1,2,0,0,0,0


In [62]:
aggregated['rutube_video_id_get_mode'] =  aggregated['rutube_video_id_get_mode'].map(video_dict)

In [63]:
from sklearn.ensemble import IsolationForest
# Находим выбросы
def detect_outliers_isolation_forest(df, columns, contamination=0.4):
    iso = IsolationForest(contamination=contamination, random_state=42)
    preds = iso.fit_predict(df[columns])
    outliers = df[preds == -1]
    return outliers

columns = [
    'event_timestamp_timezone_count', 'event_timestamp_timezone_min', 'event_timestamp_timezone_max',
    'event_timestamp_timezone_mean', 'region_get_mode', 'label_get_mode',
    'category_get_mode', 'total_watchtime_mean', 'total_watchtime_max', "total_watchtime_min", "total_watchtime_sum",
    'rutube_video_id_get_mode', 'device_get_mode', 'author_id_get_mode', 'percent_mean', 'percent_min', "percent_max", "percent_sum", 'duration_min', 'duration_max',
    'duration_sum', 'duration_mean', 'sex', 'age_class', 'morning_sum', 'day_sum', 'evening_sum', 'night_sum']
outliers_iso = detect_outliers_isolation_forest(aggregated, columns)
print(f"Количество выбросов с Isolation Forest: {outliers_iso.shape[0]}")

Количество выбросов с Isolation Forest: 43418


In [64]:
# Удаляем выбросы
aggregated = aggregated[~aggregated["viewer_uid"].isin(outliers_iso["viewer_uid"])]

In [66]:
aggregated.corr()

,viewer_uid,event_timestamp_timezone_count,event_timestamp_timezone_min,event_timestamp_timezone_max,event_timestamp_timezone_mean,region_get_mode,label_get_mode,category_get_mode,total_watchtime_mean,total_watchtime_max,...,duration_max,duration_sum,duration_mean,age,sex,age_class,morning_sum,day_sum,evening_sum,night_sum
viewer_uid,1.000000,-0.050748,0.072093,-0.031992,0.023895,-0.015955,-0.068789,-0.164921,-0.041822,-0.067561,...,-0.079925,-0.077780,-0.065468,-0.002671,0.126777,-0.007482,-0.052880,-0.023091,-0.026336,-0.015079
event_timestamp_timezone_count,-0.050748,1.000000,-0.388076,0.442577,0.030987,0.004243,-0.033868,0.060756,-0.037815,0.348158,...,0.205065,0.899136,-0.197727,-0.042037,-0.161218,-0.036384,0.591873,0.670671,0.626961,0.389245
event_timestamp_timezone_min,0.072093,-0.388076,1.000000,0.175602,0.737005,-0.022005,-0.041158,-0.098650,-0.079860,-0.305975,...,-0.212567,-0.398647,0.013630,0.027679,0.105200,0.025749,-0.249063,-0.240275,-0.243705,-0.158248
event_timestamp_timezone_max,-0.031992,0.442577,0.175602,1.000000,0.727953,0.007154,-0.075053,-0.006905,-0.050126,0.211707,...,0.189541,0.429890,-0.043557,0.030854,-0.015654,0.030182,0.263354,0.292882,0.290016,0.162011
event_timestamp_timezone_mean,0.023895,0.030987,0.737005,0.727953,1.000000,-0.008951,-0.073609,-0.066416,-0.085558,-0.066801,...,-0.023920,0.013116,-0.023610,0.041079,0.057348,0.039746,0.008252,0.033832,0.025203,-0.002257
region_get_mode,-0.015955,0.004243,-0.022005,0.007154,-0.008951,1.000000,0.032411,0.054650,0.015926,0.025789,...,-0.007382,0.001108,-0.008392,-0.048944,-0.050371,-0.046325,0.004914,0.009078,-0.002857,-0.015970
label_get_mode,-0.068789,-0.033868,-0.041158,-0.075053,-0.073609,0.032411,1.000000,0.318725,0.125567,0.068287,...,0.085340,0.019283,0.180152,-0.056745,-0.140297,-0.055088,0.013324,-0.019465,-0.041576,-0.032344
category_get_mode,-0.164921,0.060756,-0.098650,-0.006905,-0.066416,0.054650,0.318725,1.000000,0.102931,0.071128,...,0.072491,0.100614,0.134108,-0.090136,-0.291301,-0.089689,0.079939,0.034686,0.014226,0.013557
total_watchtime_mean,-0.041822,-0.037815,-0.079860,-0.050126,-0.085558,0.015926,0.125567,0.102931,1.000000,0.731382,...,0.244726,0.097356,0.355213,-0.057334,-0.117389,-0.051746,-0.029160,-0.037144,-0.006772,0.001463
total_watchtime_max,-0.067561,0.348158,-0.305975,0.211707,-0.066801,0.025789,0.068287,0.071128,0.731382,1.000000,...,0.472493,0.502206,0.322866,-0.052880,-0.134243,-0.046334,0.203356,0.208069,0.237392,0.160338


In [67]:
aggregated.describe()

,viewer_uid,event_timestamp_timezone_count,event_timestamp_timezone_min,event_timestamp_timezone_max,event_timestamp_timezone_mean,region_get_mode,label_get_mode,category_get_mode,total_watchtime_mean,total_watchtime_max,...,duration_max,duration_sum,duration_mean,age,sex,age_class,morning_sum,day_sum,evening_sum,night_sum
count,6.512800e+04,65128.000000,6.512800e+04,6.512800e+04,6.512800e+04,65128.000000,65128.000000,65128.000000,65128.000000,65128.000000,...,65128.000000,65128.000000,65128.000000,65128.000000,65128.000000,65128.000000,65128.000000,65128.000000,65128.000000,65128.000000
mean,1.017635e+07,5.288125,1.717949e+09,1.718941e+09,1.718439e+09,18.199791,29.597255,12.861442,2672.259819,3443.117922,...,5601.253449,22067.893079,4426.324796,33.266644,0.371561,1.758414,1.033918,1.333927,1.388850,0.631126
std,2.223285e+05,4.736849,6.504858e+05,6.492753e+05,5.235448e+05,19.822894,8.373855,5.964847,599.110778,902.417994,...,2081.026820,18779.805248,1429.725531,8.526693,0.483226,0.811960,1.692990,1.849214,1.849843,1.366180
min,1.000000e+07,1.000000,1.717187e+09,1.717190e+09,1.717190e+09,0.000000,0.000000,0.000000,1261.000000,1261.000000,...,1265.023000,1272.811000,1265.023000,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.004082e+07,2.000000,1.717398e+09,1.718524e+09,1.718051e+09,3.000000,30.000000,12.000000,2265.416667,2702.000000,...,4022.000000,8144.676250,3191.782473,26.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.008828e+07,4.000000,1.717773e+09,1.719080e+09,1.718427e+09,10.000000,31.000000,16.000000,2630.633333,3519.000000,...,5642.240000,16415.489000,4370.276750,33.000000,0.000000,2.000000,0.000000,1.000000,1.000000,0.000000
75%,1.020783e+07,7.000000,1.718392e+09,1.719502e+09,1.718791e+09,31.000000,31.000000,16.000000,3055.925000,4262.000000,...,6780.760000,30284.205500,5525.899958,39.000000,1.000000,2.000000,1.000000,2.000000,2.000000,1.000000
max,1.114000e+07,35.000000,1.719798e+09,1.719805e+09,1.719800e+09,111.000000,49.000000,39.000000,4693.000000,4766.000000,...,23603.665000,137427.800000,9890.493333,54.000000,1.000000,3.000000,20.000000,25.000000,18.000000,15.000000


In [207]:
#aggregated.to_csv('aggregated.csv', index=False)

In [208]:
%%time
from imblearn.over_sampling import SMOTE
# Балансируем данные по полу
smote = SMOTE(random_state=42)

x_res, y_res = smote.fit_resample(aggregated.drop(['sex', 'age', 'age_class', 'viewer_uid'], axis=1), aggregated['sex'])

CPU times: total: 3.97 s
Wall time: 899 ms


In [209]:
from sklearn.model_selection import train_test_split
# Делим на выборки
x_train, x_test, y_train, y_test = train_test_split(x_res,
                                                    y_res, test_size=0.33, random_state=42)

In [210]:
from catboost import CatBoostClassifier
# Модель классификации
model = CatBoostClassifier(iterations=1500, eval_metric='TotalF1', learning_rate=0.25, max_depth=6,
                           early_stopping_rounds=250, task_type="GPU")

In [211]:
model.fit(x_train, y_train, eval_set=(x_test, y_test),cat_features=['region_get_mode', 'author_id_get_mode', 'rutube_video_id_get_mode'], use_best_model=True)

0:	learn: 0.7994386	test: 0.7968913	best: 0.7968913 (0)	total: 49.3ms	remaining: 1m 13s
1:	learn: 0.7995745	test: 0.7967867	best: 0.7968913 (0)	total: 82.1ms	remaining: 1m 1s
2:	learn: 0.8013724	test: 0.7991089	best: 0.7991089 (2)	total: 115ms	remaining: 57.2s
3:	learn: 0.8031163	test: 0.8005366	best: 0.8005366 (3)	total: 238ms	remaining: 1m 28s
4:	learn: 0.8071165	test: 0.8044209	best: 0.8044209 (4)	total: 316ms	remaining: 1m 34s
5:	learn: 0.8077419	test: 0.8053114	best: 0.8053114 (5)	total: 351ms	remaining: 1m 27s
6:	learn: 0.8106837	test: 0.8068999	best: 0.8068999 (6)	total: 386ms	remaining: 1m 22s
7:	learn: 0.8108778	test: 0.8067454	best: 0.8068999 (6)	total: 419ms	remaining: 1m 18s
8:	learn: 0.8173885	test: 0.8146712	best: 0.8146712 (8)	total: 444ms	remaining: 1m 13s
9:	learn: 0.8185473	test: 0.8151435	best: 0.8151435 (9)	total: 469ms	remaining: 1m 9s
10:	learn: 0.8194338	test: 0.8171855	best: 0.8171855 (10)	total: 503ms	remaining: 1m 8s
11:	learn: 0.8201524	test: 0.8182890	best: 

In [212]:
#model.save_model('sex_model')

In [213]:
y_pred = model.predict(x_test)

In [214]:
from sklearn.metrics import f1_score, accuracy_score

acc = f1_score(y_test, y_pred, average='weighted')
acc

0.8271057964022422

In [215]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,author_id_get_mode,35.000963
1,rutube_video_id_get_mode,7.097753
2,region_get_mode,4.325385
3,duration_min,4.240913
4,device_get_mode,3.950040
5,category_get_mode,3.880949
6,label_get_mode,3.733173
7,percent_max,3.522933
8,duration_max,3.179471
9,event_timestamp_timezone_min,2.604940


In [216]:
%%time
from imblearn.over_sampling import SMOTE
# Балансируем данные по возрастному бакету
smote = SMOTE(random_state=42)

x_res, y_res = smote.fit_resample(aggregated.drop(['sex', 'age', 'age_class', 'viewer_uid'], axis=1), aggregated['age_class'])

CPU times: total: 5.92 s
Wall time: 1.45 s


In [217]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_res,
                                                    y_res, test_size=0.33, random_state=42)

In [218]:
from catboost import CatBoostClassifier
# Модель классификации
model = CatBoostClassifier(iterations=2500, eval_metric='TotalF1', learning_rate=0.1, max_depth=6,
                           early_stopping_rounds=250,  task_type="GPU", leaf_estimation_backtracking='AnyImprovement', loss_function='MultiClass')

In [219]:
model.fit(x_train, y_train, eval_set=(x_test, y_test), use_best_model=True) # cat_features=['region_get_mode', 'author_id_get_mode', 'rutube_video_id_get_mode']

0:	learn: 0.3987389	test: 0.3978832	best: 0.3978832 (0)	total: 38ms	remaining: 1m 35s
1:	learn: 0.4028552	test: 0.4045914	best: 0.4045914 (1)	total: 48.9ms	remaining: 1m 1s
2:	learn: 0.4012234	test: 0.4026235	best: 0.4045914 (1)	total: 61.4ms	remaining: 51.1s
3:	learn: 0.4090974	test: 0.4093889	best: 0.4093889 (3)	total: 71.8ms	remaining: 44.8s
4:	learn: 0.4184486	test: 0.4172850	best: 0.4172850 (4)	total: 83.2ms	remaining: 41.5s
5:	learn: 0.4201407	test: 0.4189142	best: 0.4189142 (5)	total: 102ms	remaining: 42.4s
6:	learn: 0.4233085	test: 0.4232601	best: 0.4232601 (6)	total: 130ms	remaining: 46.3s
7:	learn: 0.4272036	test: 0.4259526	best: 0.4259526 (7)	total: 154ms	remaining: 48.1s
8:	learn: 0.4324980	test: 0.4308756	best: 0.4308756 (8)	total: 180ms	remaining: 49.9s
9:	learn: 0.4347575	test: 0.4341010	best: 0.4341010 (9)	total: 206ms	remaining: 51.3s
10:	learn: 0.4386448	test: 0.4372570	best: 0.4372570 (10)	total: 234ms	remaining: 52.9s
11:	learn: 0.4441369	test: 0.4441594	best: 0.444

In [220]:
y_pred = model.predict(x_test)

In [221]:
f = f1_score(y_test, y_pred, average='weighted')
f

0.5899754726468687

In [222]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,author_id_get_mode,11.354947
1,rutube_video_id_get_mode,6.091383
2,label_get_mode,5.937290
3,event_timestamp_timezone_max,5.287377
4,duration_min,5.018028
5,event_timestamp_timezone_min,4.968720
6,region_get_mode,4.917844
7,event_timestamp_timezone_mean,4.755795
8,category_get_mode,4.728948
9,duration_max,4.558634


In [223]:
0.7 * f + 0.3 * acc

0.6611145697734807

In [224]:
model.save_model("age_model")